In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from  matplotlib import pyplot as plt

In [2]:
data_path = '../input/mvsasingle/MVSA_Single/data'
label_path = '../input/mvsasingle/MVSA_Single/labelResultAll.txt'

In [3]:
def read_text_file(path):
    with open(path, 'r', encoding='latin-1') as f:
        return f.read()
    
def read_image_file(path):
    image = cv2.imread(path, cv2.COLOR_BGR2RGB)
    return image

def read_labels_file(path):
    df = pd.read_csv(path, sep="\s+|,", engine="python")
    return df

In [4]:
# IMAGE_SIZE = (224, 224)
# def read_image_file(path):
#     image = cv2.imread(path, cv2.COLOR_BGR2RGB)
# #     image = np.array(image)#.astype('float32')
# #     image = cv2.resize(image, IMAGE_SIZE, interpolation = cv2.INTER_AREA)
# #     image /= 255
# #     print(image.shape)
# #     ax = plt.subplot(1,2,1)
# #     plt.imshow(image)
# #     image = cv2.resize(image, IMAGE_SIZE, interpolation = cv2.INTER_AREA)
# #     ax = plt.subplot(1,2,2)
# #     plt.imshow(image)
# #     print(image.shape)
#     return image
# read_image_file('../input/mvsasingle/MVSA_Single/data/10.jpg')      

In [5]:
def ground_truth_label(text_label, image_label):
    if text_label == image_label == 'positive':
        label = 'positive'
    elif text_label == image_label == 'negative':
        label = 'negative'
    elif text_label == image_label == 'neutral':
        label = 'neutral'
    elif (text_label == 'neutral' and image_label != 'neutral') or (text_label != 'neutral' or image_label == 'neutral'):
        label = image_label if text_label == 'neutral' else text_label
    else:# (text_label == 'positive' and image_label == 'negative') or (text_label == 'negative' and image_label == 'positive'):
        label = 'remove'
    return label

def create_labels(path):
    dataframe = read_labels_file(path)
    labels = []
    for _, row in dataframe.iterrows():
        label = ground_truth_label(row["text"], row["image"])
        labels.append(label)
    return np.array(labels, dtype='object')

In [10]:
def create_data(path):
    texts = []
    images = []
    
    # read data along with its filename as ID
    for filename in tqdm(os.listdir(data_path)):
        ID = int(filename.split('.')[0])
        file_path = os.path.join(path, filename)
        if filename.endswith('txt'):
            text = read_text_file(file_path)
            texts.append((ID, text))
        else:
            image = read_image_file(file_path)
            images.append((ID, image))
    
    # Sort data by its ID
    get_ID = lambda x : x[0]
    texts.sort(key=get_ID)
    images.sort(key=get_ID)
    
    # return data without ID
    texts = np.array([text[1] for text in texts])
    images = np.array([image[1] for image in images])

    return texts, images

In [11]:
texts, images = create_data(data_path)
labels = create_labels(label_path)

100%|██████████| 9738/9738 [00:30<00:00, 318.46it/s]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
# df_labels = read_labels_file('../input/mvsasingle/MVSA_Single/labelResultAll.txt')
# df_labels['text'].count

In [ ]:
# label_text_count = df_labels.value_counts('text')
# label_image_count = df_labels.value_counts('image')
# label_text_count

In [ ]:
# text_ids = []
# image_ids = []
# for i in range(len(texts)):
#     if texts[i][0] != images[i][0] != read_labels_file(label_path)['ID'][i]:
#         print('here')

In [ ]:
u, c, i = np.unique(texts, return_index=True, return_counts=True, axis=1)
dup = u[c > 1]
u[:10]

In [ ]:
remove_indices = [i for i in range(labels.shape[0]) if labels[i] == 'remove'] # labels that conflicts between image and text
# np.delete......

In [ ]:
def display_sample(text, image):
    plt.imshow(image)
    print('Text:', text)
    plt.show()
display_sample(texts_with_id[2][1], images_with_id[2][1])

In [ ]:
shapes = []
for image in images_with_id:
    shapes.append(image.shape)
shapes

In [ ]:
raw_texts[raw_texts.duplicated('Text')]

In [ ]:
len_split = lambda x: len(x.split())
print()
print('Min number of words in text:', raw_texts['Text'].apply(len_split).min())
print('Max number of words in text:', raw_texts['Text'].apply(len_split).max())
print('Average number of words in text:', round(raw_texts['Text'].apply(len_split).mean()))